You have to work on the files:
*  [Books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz)
*  [Book ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz)
*  [Users](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz)
*  [Goodbooks books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz)
*  [Goodbooks ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz)

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    To read those files, you need to use the `encoding = 'latin-1'` option.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com), otherwise no  answer will be given.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
#import time
#start = time.time()
import pandas as pd
import numpy as np
import re

In [3]:
users = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz', compression = 'gzip', index_col = 'User-ID', encoding = 'latin-1', sep = ';', dtype = {'User-ID': 'int',
                                                                                                                                                                                         'Location': 'string',
                                                                                                                                                                                         'Age': 'Int64'
                                                                                                                                                                                        })
users.shape

#gli index_col servono o no?

(278858, 2)

In [4]:
books = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz', compression = 'gzip', escapechar="\\", encoding = 'latin-1', sep = ';', dtype = {'ISBN': 'string',
                                                                                                                                                                                  'Book-title': 'string',
                                                                                                                                                                                  'Book-Author': 'string',
                                                                                                                                                                                  'Year-Of-Publication': 'int',
                                                                                                                                                                                  'Publisher': 'string',
                                                                                                                                                                                  'Image-URL-S': 'string',
                                                                                                                                                                                  'Image-URL-M': 'string',
                                                                                                                                                                                  'Image-URL-L': 'string',
                                                                                                                                                                                 })
books.shape

(271359, 8)

In [5]:
book_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz', compression = 'gzip', index_col = ['User-ID', 'ISBN'], encoding = 'latin-1', sep = ';', dtype = {'User-ID': 'int',
                                                                                                                                                                                                                 'ISBN': 'string',
                                                                                                                                                                                                                 'Book-Rating': 'int'
                                                                                                                                                                                                                })
book_ratings.shape

# 0 significa che non hanno votato?
# warning 

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(1149780, 1)

In [6]:
goodbooks_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz', compression = 'gzip', encoding = 'latin-1', sep = ',', dtype = {'user_id': 'int',
                                                                                                                                                                                        'book_id': 'int',
                                                                                                                                                                                        'rating': 'int'
                                                                                                                                                                                        })
goodbooks_ratings.shape

# index_col = ['user_id', 'book_id'],
#anno -720 é un NA

(99, 3)

In [7]:
goodbooks = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz', compression = 'gzip', encoding = 'latin-1', sep = ',', dtype = {'book_id': 'int',
                                                                                                                                                                         'goodreads_book_id': 'int',
                                                                                                                                                                         'best_book_id': 'int',
                                                                                                                                                                         'work_id': 'int',
                                                                                                                                                                         'books_count': 'int',
                                                                                                                                                                         'isbn': 'string',
                                                                                                                                                                         'isbn13': 'string',
                                                                                                                                                                         'authors': 'string',
                                                                                                                                                                         'original_publication_year': 'int',
                                                                                                                                                                         'original_title': 'string',
                                                                                                                                                                         'title': 'string',
                                                                                                                                                                         'language_code': 'string',
                                                                                                                                                                         'average_rating': 'double',
                                                                                                                                                                         'ratings_count': 'int',
                                                                                                                                                                         'work_ratings_count': 'int',
                                                                                                                                                                         'work_text_reviews_count': 'int',
                                                                                                                                                                         'ratings_1': 'int',
                                                                                                                                                                         'ratings_2': 'int',
                                                                                                                                                                         'ratings_3': 'int',
                                                                                                                                                                         'ratings_4': 'int',
                                                                                                                                                                         'ratings_5': 'int',
                                                                                                                                                                         'image_url': 'string',
                                                                                                                                                                         'small_image_url': 'string'
                                                                                                                                                                         })
goodbooks.shape

# index_col = 'book_id',

(99, 23)

### 1. Normalize the location field of *Users* dataset, splitting into city, region, country.

In [8]:
users[['City', 'Region', 'Country']] = users['Location'].str.rsplit(',', 2, expand=True)
users.head()

# Da sistemare country usando le probabilitá (?)
# creare lista 
# sistemare encoding (riga 1318)

,Location,Age,City,Region,Country
User-ID,,,,,
1,"nyc, new york, usa",<NA>,nyc,new york,usa
2,"stockton, california, usa",18,stockton,california,usa
3,"moscow, yukon territory, russia",<NA>,moscow,yukon territory,russia
4,"porto, v.n.gaia, portugal",17,porto,v.n.gaia,portugal
5,"farnborough, hants, united kingdom",<NA>,farnborough,hants,united kingdom


### 2. For each book in the *Books* dataset, compute its average rating.

In [12]:
pd.merge(books, book_ratings, on = 'ISBN', how = 'left').groupby('ISBN', as_index=False).agg(Average_Rating=('Book-Rating', 'mean'))

# gli 0 vanno mediati?

,ISBN,Average_Rating
0,0000913154,8.0
1,0001010565,0.0
2,0001046438,9.0
3,0001046713,0.0
4,000104687X,6.0
...,...,...
271354,B000234N76,0.0
271355,B000234NC6,0.0
271356,B00029DGGO,0.0
271357,B0002JV9PY,0.0


### 3. For each book in the *GoodBooks* dataset, compute its average rating.

In [13]:
goodbooks[['isbn', 'average_rating']]

,isbn,average_rating
0,439023483,4.34
1,439554934,4.44
2,316015849,3.57
3,61120081,4.25
4,743273567,3.89
...,...,...
94,375751513,4.06
95,345803507,3.88
96,393970124,3.98
97,307269981,4.22


### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset `merged books`. The books that have not been merged together will not appear in `merged books`.

In [116]:
merging_books = books[books.duplicated(subset = ['Book-Title', 'Book-Author', 'Publisher'], keep = False)].sort_values(['Book-Title', 'Book-Author', 'Publisher'])
merged_books = merging_books.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False).agg(Count=('ISBN', 'count'))
merged_books.head()

,Book-Title,Book-Author,Publisher,Count
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,2


### 5. For each book in `merged books` compute its average rating.

The average is computed considering all books in `books` that have been merged.

In [93]:
book_ratings_avg = book_ratings.groupby('ISBN', as_index=True).agg(Average_Rating=('Book-Rating', 'mean'))
merging_book_ratings = pd.merge(merging_books, book_ratings_avg, on = 'ISBN', how = 'left')
merging_book_ratings.head()


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Average_Rating
0,1565920465,!%@ (A Nutshell handbook),Donnalyn Frey,1994,O'Reilly,http://images.amazon.com/images/P/1565920465.0...,http://images.amazon.com/images/P/1565920465.0...,http://images.amazon.com/images/P/1565920465.0...,0.00
1,1565920317,!%@ (A Nutshell handbook),Donnalyn Frey,1993,O'Reilly,http://images.amazon.com/images/P/1565920317.0...,http://images.amazon.com/images/P/1565920317.0...,http://images.amazon.com/images/P/1565920317.0...,6.00
2,0792276833,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,2000,National Geographic,http://images.amazon.com/images/P/0792276833.0...,http://images.amazon.com/images/P/0792276833.0...,http://images.amazon.com/images/P/0792276833.0...,0.00
3,0792277295,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,2001,National Geographic,http://images.amazon.com/images/P/0792277295.0...,http://images.amazon.com/images/P/0792277295.0...,http://images.amazon.com/images/P/0792277295.0...,3.40
4,1566196256,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,1995,Barnes &amp; Nobles Books,http://images.amazon.com/images/P/1566196256.0...,http://images.amazon.com/images/P/1566196256.0...,http://images.amazon.com/images/P/1566196256.0...,7.25


### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

In [117]:
merging_book_ratings.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False).agg({'Average_Rating' : ['min', 'max']}).head()

Book-Title              Book-Author  \
                                                                                
0                          !%@ (A Nutshell handbook)            Donnalyn Frey   
1  'A Hell of a Place to Lose a Cow': An American...              Tim Brookes   
2  10,000 dreams interpreted: A dictionary of dreams  Gustavus Hindman Miller   
3                                   101 Famous Poems              Roy J. Cook   
4                 15 Houseplants Even You Can't Kill                Joe Elder   

                        Publisher Average_Rating        
                                             min   max  
0                        O'Reilly       0.000000  6.00  
1             National Geographic       0.000000  3.40  
2       Barnes &amp; Nobles Books       6.666667  7.25  
3  McGraw-Hill/Contemporary Books       0.000000  5.00  
4               Berkley Pub Group       0.000000  0.00

### 7. For each book in `goodbooks`, compute the list of its authors. Assuming that the number of reviews with a text (column `work_text_reviews_count`) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the *shared number of reviews with a text*.

In [110]:
goodbooks['authors_splitted'] = goodbooks['authors'].str.split(',')
goodbooks['shared_number_of_reviews_with_a_text'] = goodbooks['work_text_reviews_count'] / goodbooks['authors_splitted'].apply(len)

goodbooks_exploded = goodbooks.explode('authors_splitted')
goodbooks_exploded.groupby(['authors_splitted'], as_index=False).sum('shared_number_of_reviews_with_a_text').head() [['authors_splitted', 'shared_number_of_reviews_with_a_text']]

,authors_splitted,shared_number_of_reviews_with_a_text
0,Alan R. Clarke,27890.500000
1,B.M. Mooyaart-Doubleday,6941.666667
2,Bernard Knox,1620.200000
3,CelÃ¢l Ãster,15172.666667
4,David J. Skal,5754.333333


### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

In [79]:
temp = goodbooks_exploded.groupby(['original_publication_year', 'authors_splitted'], as_index=False).sum()
temp.loc[temp.groupby('original_publication_year')['shared_number_of_reviews_with_a_text'].idxmax()].head() [['original_publication_year', 'authors_splitted', 'shared_number_of_reviews_with_a_text']]

,original_publication_year,authors_splitted,shared_number_of_reviews_with_a_text
0,-720,Bernard Knox,1620.200000
5,1595,Robert Jackson,7389.000000
7,1811,Ros Ballaster,3842.000000
10,1813,Jane Austen,49152.000000
11,1818,Maurice Hindle,6664.333333


### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

In [45]:
aaa = pd.merge(books, goodbooks, left_on = 'ISBN', right_on = 'isbn')
aaa = pd.merge(aaa, book_ratings, on = 'ISBN', how = 'left')
aaa = pd.merge(aaa, goodbooks_ratings, on = 'book_id', how = 'left').groupby('ISBN', as_index=False).agg(Book_Average_Rating=('Book-Rating', 'mean'), Goodbook_Average_Rating=('rating', 'mean'))
aaa['Difference'] = aaa['Book_Average_Rating']/2 - aaa['Goodbook_Average_Rating']
aaa


# da controllare
#differenza dei scala voti (5 dei goodbooks e 10 dei books)

,ISBN,Book_Average_Rating,Goodbook_Average_Rating,Difference
0,014028009X,3.751852,NaN,NaN
1,014038572X,4.460674,NaN,NaN
2,043965548X,3.533333,5.0,-3.233333
3,1400032717,4.813187,NaN,NaN
4,1594480001,2.400000,NaN,NaN


### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

In [107]:
users.groupby(pd.cut(users['Age'], np.arange(14, 150, 10))).count()

#da fare
#come trattare le etá sbagliate (> 100 anni)?
#come ultimo bin affidabile terrei 84-94

,Location,Age,City,Region,Country
Age,,,,,
"(14, 24]",40001,40001,40001,40001,40001
"(24, 34]",50767,50767,50767,50766,50766
"(34, 44]",32690,32690,32690,32690,32690
"(44, 54]",23152,23152,23152,23152,23152
"(54, 64]",12493,12493,12493,12493,12493
"(64, 74]",3596,3596,3596,3596,3595
"(74, 84]",615,615,615,615,615
"(84, 94]",83,83,83,83,83
"(94, 104]",278,278,278,278,278


In [ ]:
#users_UnknownAge_and_Outliers = 



### 11. Find the books that appear only in the goodbooks datasets.

In [ ]:
# é esattamente il goodbooks dataset?

### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [115]:
prova2 = books.groupby(['Book-Author', 'Book-Title'], as_index=False).count()
prova2.sort_values('ISBN')

#da sistemare!!! LITTLE WOMEN

,Book-Author,Book-Title,ISBN,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,D. Chiel,Die Hard: With a Vengeance - A Novel,1,1,1,1,1,1
164746,Mary Pope Osborne,"Afternoon on the Amazon (Magic Tree House, # 6)",1,1,1,1,1,1
164747,Mary Pope Osborne,Beauty and the Beast,1,1,1,1,1,1
164748,Mary Pope Osborne,"Best Wishes, Joe Brady",1,1,1,1,1,1
164749,Mary Pope Osborne,"Buffalo Before Breakfast (Magic Tree House 18,...",1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
36524,Charles Dickens,Great Expectations,16,16,16,16,16,16
106557,Jane Austen,Pride and Prejudice,18,18,18,18,18,18
161311,Mark Twain,Adventures of Huckleberry Finn,20,20,20,20,20,20
69480,Emily Bronte,Wuthering Heights,20,20,20,20,20,20


### 13. Find the author with the highest average rating according to the goodbooks datasets.

In [77]:
prova = goodbooks_exploded.groupby('authors_splitted', as_index=False).mean('average_rating')
prova.loc[prova['average_rating'] == max(prova['average_rating'])] [['authors_splitted', 'average_rating']]

,authors_splitted,average_rating
32,Rufus Beck,4.53
